# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [64]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys
import json

sys.path.append('../../')
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [65]:
city=pd.read_csv("../WeatherPy/output_data/cities.csv")
city=city.drop(columns="Unnamed: 0")
city

,city,lat,lon,temp,humidity,cloudiness,windspeed
0,punta arenas,-53.1500,-70.9167,9.06,66,40,5.14
1,cockburn town,21.4612,-71.1419,25.26,75,0,6.13
2,mar del plata,-38.0023,-57.5575,21.72,91,0,3.13
3,imbituba,-28.2400,-48.6703,26.57,86,73,4.61
4,san quintin,30.4833,-115.9500,14.08,62,2,1.07
5,victoria,22.2855,114.1577,22.32,86,100,0.89
6,busselton,-33.6500,115.3333,27.27,40,0,9.23
7,bangassou,4.7413,22.8184,33.80,17,10,1.95
8,thompson,55.7435,-97.8558,-29.91,69,0,4.12
9,orlik,52.5178,99.8250,-14.93,89,100,0.15


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [66]:
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = city[["lat", "lon"]].astype(float)


In [67]:
%matplotlib notebook
humidity = city["humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
#hotel_df = city.drop(city["temp"] < 10.0 & city["temp"] > 30.0)
#city["temp"].values
city=city.where(city["temp"] >= 20).dropna()
city
#.columns[city.apply(lambda col: col.isnull().sum() > 3)], axis=1)

,city,lat,lon,temp,humidity,cloudiness,windspeed
1,cockburn town,21.4612,-71.1419,25.26,75.0,0.0,6.13
2,mar del plata,-38.0023,-57.5575,21.72,91.0,0.0,3.13
3,imbituba,-28.2400,-48.6703,26.57,86.0,73.0,4.61
5,victoria,22.2855,114.1577,22.32,86.0,100.0,0.89
6,busselton,-33.6500,115.3333,27.27,40.0,0.0,9.23
7,bangassou,4.7413,22.8184,33.80,17.0,10.0,1.95
12,hobart,-42.8794,147.3294,22.37,75.0,75.0,0.45
17,airai,-8.9266,125.4092,20.25,78.0,100.0,1.04
19,cairns,-16.9167,145.7667,28.97,73.0,100.0,4.12
24,watsa,3.0372,29.5355,29.06,31.0,56.0,2.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
hotel_df=city
hotel_df.loc[:,"Hotel Name"]=""
hotel_df.loc[:,"Country"]=""
#hotel_df
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        words = name_address["results"][0]["plus_code"]["compound_code"].split(",")
        print(words)
        hotel_df.loc[index, "Country"] = words[-1]
#        cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
#        cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

['FV53+R2 Cockburn Town', ' Turks and Caicos Islands']
['XF96+74 Mar del Plata', ' Buenos Aires Province', ' Argentina']
['Q83M+96 Centro', ' Imbituba - State of Santa Catarina', ' Brazil']
['75XG+R4 Tsim Sha Tsui', ' Hong Kong']
['87RG+HM Broadwater', ' Western Australia', ' Australia']
Missing field/result... skipping.
['38WP+WW Sandy Bay', ' Tasmania', ' Australia']
Missing field/result... skipping.
['3QR5+QV Cairns City', ' Queensland', ' Australia']
Missing field/result... skipping.


In [70]:
 hotel_df

,city,lat,lon,temp,humidity,cloudiness,windspeed,Hotel Name,Country
1,cockburn town,21.4612,-71.1419,25.26,75.0,0.0,6.13,Osprey Beach Hotel,Turks and Caicos Islands
2,mar del plata,-38.0023,-57.5575,21.72,91.0,0.0,3.13,Sheraton Mar del Plata Hotel,Argentina
3,imbituba,-28.2400,-48.6703,26.57,86.0,73.0,4.61,Praia Hotel Imbituba,Brazil
5,victoria,22.2855,114.1577,22.32,86.0,100.0,0.89,Park Hotel Hong Kong,Hong Kong
6,busselton,-33.6500,115.3333,27.27,40.0,0.0,9.23,Bayview Geographe Resort,Australia
7,bangassou,4.7413,22.8184,33.80,17.0,10.0,1.95,,
12,hobart,-42.8794,147.3294,22.37,75.0,75.0,0.45,Wrest Point,Australia
17,airai,-8.9266,125.4092,20.25,78.0,100.0,1.04,,
19,cairns,-16.9167,145.7667,28.97,73.0,100.0,4.12,Cairns Sheridan Hotel (When reviewing do not c...,Australia
24,watsa,3.0372,29.5355,29.06,31.0,56.0,2.10,,


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]
hotel_df
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Osprey Beach Hotel</dd>\n<dt>City</dt><dd>cockburn town</dd>\n<dt>Country</dt><dd> Turks and Caicos Islands</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Sheraton Mar del Plata Hotel</dd>\n<dt>City</dt><dd>mar del plata</dd>\n<dt>Country</dt><dd> Argentina</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Praia Hotel Imbituba</dd>\n<dt>City</dt><dd>imbituba</dd>\n<dt>Country</dt><dd> Brazil</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Park Hotel Hong Kong</dd>\n<dt>City</dt><dd>victoria</dd>\n<dt>Country</dt><dd> Hong Kong</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Bayview Geographe Resort</dd>\n<dt>City</dt><dd>busselton</dd>\n<dt>Country</dt><dd> Australia</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>bangassou</dd>\n<dt>Country</dt><dd></dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Wrest Point</dd>\n<dt>City</dt><dd>hobart</dd>\n<dt>Country</dt><dd> Australia</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>airai</dd>\n<dt>Country</

In [73]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content=[f"Hotel Info {row}" for row in hotel_info])
# Display figure
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))